In [ ]:
%matplotlib inline
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
from pandas_plink import read_plink#1_bin
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from joblib import Parallel, delayed
from sklearn import metrics
from math import sqrt
import multiprocessing
from multiprocessing import Pool
import random
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn import metrics
import os
import time


In [ ]:
print(os.environ['CONDA_DEFAULT_ENV'])
os.environ["CUDA_VISIBLE_DEVICES"]="1"
'''To be changed'''
path_for_files="/work/biobank/ukb_data/new_data/"

In [ ]:
'''Get genotype SNPs data'''
(bim,fam,bed)=read_plink(path_for_files+"all")

In [ ]:
'''Get phenotypes (cancer here)'''
Yd1=pd.read_csv(path_for_files+"cancer_pheno.csv",sep=",")
Yd2=pd.read_csv(path_for_files+"non_cancer_pheno.csv",sep=",")


In [ ]:
'''get intersection of individuals'''
ids=list(set(fam["iid"].astype("int64")).intersection(set(Yd1["Unnamed: 0"])))
ids.sort()
'''Get intersecting individuals in phenotype and concatenate the disease phenotypes with cancer phnotypes to obtain the whole Y'''
Yd1=Yd1.set_index('Unnamed: 0').loc[ids].values
Yd2=Yd2.set_index('Unnamed: 0').loc[ids].values
Yd=np.concatenate([Yd1,Yd2],axis=1)

In [ ]:
Yd.shape

In [ ]:
'''Reduce size of genotype dataset with convertion in uint8'''
print("Convertion")
bed=bed.astype('uint8')
bed=bed.T

In [ ]:
'''get the matching genotypes wrt phenotypes'''
bed=bed[ids]

In [ ]:
'''Divide xarrays for training/validation/test sets'''
x_train=bed[:bed.shape[0]*70//100,:]
x_val=bed[bed.shape[0]*70//100:bed.shape[0]*85//100,:]
x_test=bed[bed.shape[0]*85//100:,:]
y_train=Yd[:bed.shape[0]*70//100,:]
assert x_train.shape[0]==y_train.shape[0]
y_val=Yd[bed.shape[0]*70//100:bed.shape[0]*85//100,:]
assert x_val.shape[0]==y_val.shape[0]
y_test=Yd[bed.shape[0]*85//100:,:]
assert x_test.shape[0]==y_test.shape[0]

In [ ]:
'''Model'''
tf.compat.v1.reset_default_graph()
X=tf.placeholder(tf.float32,shape=(None,x_train.shape[1]),name="X")
Y=tf.placeholder(tf.float32,shape=(None,y_train.shape[1]),name="Y")
with tf.name_scope("dnn"):
    training = tf.placeholder_with_default(False, shape=(), name='training')
    initializer = tf.compat.v1.initializers.he_normal()
    hidden00_drop= tf.layers.dropout(X, 0.5, training=training)
    hidden01=tf.layers.dense(hidden00_drop, 1000, name="hidden01",activation=None, kernel_initializer=initializer)
    hidden01_norm=tf.layers.batch_normalization(hidden01, training=training, momentum=0.9)
    act_hidden01=tf.nn.leaky_relu(hidden01_norm)
    hidden01_drop = tf.layers.dropout(act_hidden01, 0.5, training=training)
    hidden0=tf.layers.dense(hidden01_drop, 200, name="hidden0",activation=None, kernel_initializer=initializer)
    hidden0_norm=tf.layers.batch_normalization(hidden0, training=training, momentum=0.9)
    act_hidden0=tf.nn.leaky_relu(hidden0_norm)
    hidden0_drop = tf.layers.dropout(act_hidden0, 0.5, training=training)
    hidden1=tf.layers.dense(hidden0_drop, 50, name="hidden1",activation=None, kernel_initializer=initializer)
    hidden1_norm=tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
    act_hidden1=tf.nn.leaky_relu(hidden1_norm)
    dp=tf.layers.dropout( act_hidden1, 0.5, training=training)
    A=tf.layers.dense(hidden1_norm, x_train.shape[1]+1, name="A",activation=None, kernel_initializer=initializer)
    M=tf.multiply(A,X)
    M_drop = tf.layers.dropout( M, 0.5, training=training)
    weights = tf.trainable_variables()[-2]
    output=tf.layers.dense(  M_drop, y_train.shape[1], name="output_final",activation=None)

In [ ]:
'''loss and L2 regularization'''
with tf.name_scope("loss"):
    l2_regularizer = tf.keras.regularizers.L2(0.001)
    cross_entropy =tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=output)
    error=tf.reduce_mean(cross_entropy)+l2_regularizer(weights)

In [ ]:
'''Adam optimizer'''
with tf.name_scope("train"):
    optimizer =tf.train.AdamOptimizer(learning_rate=0.0001,beta1=0.9,beta2=0.999,epsilon=1e-08,use_locking=False,name='Adam')
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_update_ops):
        training_op = optimizer.minimize(error)

In [ ]:
'''metrics by tensorflow if needed + output activation'''
with tf.name_scope("eval"):
    predicted = tf.nn.sigmoid(output)
    correct_pred = tf.equal(tf.round(predicted), Y)
    #acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    #_,auc = tf.metrics.auc(labels=Y[:,1],predictions=predicted[:,1])

In [ ]:
'''Reinitialize session if needed'''
try:
    sess.close()
except:
    pass

In [ ]:
'''Initialize training'''
init=tf.global_variables_initializer()
loc=tf.local_variables_initializer()
sess = tf.InteractiveSession(config=tf.ConfigProto(device_count={ "GPU": 1}))
init.run()
loc.run()
accuracy_tab=[]
loss_tab=[]
epoch_tab=[]
accuracy_tab_val=[]
loss_tab_val=[]

In [ ]:
'''Mini Batch function'''
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [ ]:
saver=tf.train.Saver()

In [ ]:
'''Training heuristic'''
start = time.time()
batch_size=512
best_loss=7777
size_big_batch=120000
size_big_val_test=50000
open(path_for_files+'Testc822_mtl_362_l2_dropout.txt', 'w')
for epoch in range(200):
    iteration=0
    '''Perform training iteration'''
    for i in range(x_train.shape[0]//size_big_batch+1):
        x_train_big=x_train[size_big_batch*i:size_big_batch*(i+1),:].compute()
        y_train_big=y_train[size_big_batch*i:size_big_batch*(i+1),:]
        for x_batch,y_batch in shuffle_batch(x_train_big,y_train_big, batch_size):#,ones,zeros):
            loc.run()
            x_batch[np.isnan(x_batch)]=2
            sess.run(training_op,feed_dict={X:x_batch,Y:y_batch,W:w,training:True})
            print("%d ITERATION:%d/%d "%(epoch,iteration,len(x_train)//batch_size),end='\r')
            iteration+=1
        del x_train_big
    loc.run()
    loss_train=sess.run(error,feed_dict={X:x_batcht,Y:y_batch,W:w,training:False})
    print(epoch,"Batch Train Loss:",loss_train)
    epoch_tab.append(epoch)
    loss_tab.append(loss_train)
    '''Compute validation scores'''
    loss_values=[]
    predicted_values=[]
    for j in range(x_val.shape[0]//size_big_val_test+1):
        loc.run()
        x_val_big=x_val[size_big_val_test*j:size_big_val_test*(j+1),:].compute()
        x_val_big[np.isnan(x_val_big)]=2
        y_val_big=y_val[size_big_val_test*j:size_big_val_test*(j+1),:]
        for i in range(x_val_big.shape[0]//batch_size+1):
            x_batch_val=x_val_big[i*batch_size:(i+1)*batch_size]
            loss_val_loc,predicted_val_loc=sess.run([error,predicted],feed_dict={X:x_batch_val,Y:y_val_big[i*batch_size:(i+1)*batch_size,:],W:w,training:False})
            predicted_values.append(predicted_val_loc)
            loss_values.append(loss_val_loc)
        del x_val_big
    loss_val=np.mean(loss_values)
    '''Save results if improves best score so far'''
    if loss_val<best_loss:
        save_path = saver.save(sess, path_for_files+"modelc822_mtl_362_l2_dropout_loss.ckpt")
        best_loss=loss_val
    with open(path_for_files+'Testc822_mtl_362_l2_dropout.txt', 'a+') as file:
        file.write("Epoch"+str(epoch)+" Loss:"+str(loss_val)+"\n\n")
    print(epoch,"Loss val:",loss_val)
    print("\n")
end = time.time()
print(end - start)

In [ ]:
'''Get best model for final test metrics evaluation'''
sess.close()
sess = tf.InteractiveSession(config=tf.ConfigProto(device_count={ "GPU": 1}))
saver.restore(sess, path_for_files+"modelc822_mtl_362_l2_dropout_loss.ckpt")

In [ ]:
'''predict test prediction from the model'''
pred_test=[]
pred_test=[]
size_big_val_test=35000
batch_size=512
for j in range(x_test.shape[0]//size_big_val_test+1):
    loc.run()
    x_test_big=x_test[size_big_val_test*j:size_big_val_test*(j+1),:].compute()
    x_test_big[np.isnan(x_test_big)]=2
    y_test_big=y_test[size_big_val_test*j:size_big_val_test*(j+1),:]
    for i in range(x_test_big.shape[0]//batch_size+1):
        x_batch_test=x_test_big[i*batch_size:(i+1)*batch_size]
        predicted_loc=sess.run(predicted,feed_dict={X:x_batch_test,training:False})
        pred_test.append(predicted_loc)
    del x_test_big

In [ ]:
'''Compute test metrics and store them in a dictionnary disease name--> list of metrics'''
from sklearn import metrics
pred_test_v=np.concatenate(pred_test,axis=0)
diseases=pd.read_csv(path_for_files+"cancer_pheno.csv",sep=",").drop(columns='Unnamed: 0').columns
dic_res={}
for i in range(len(y_test.shape[1])):
    fpr, tpr, thresholds = metrics.roc_curve(y_test[:,i],pred_test_v[:,i], pos_label=1)
    precision, recall, thresholds = metrics.precision_recall_curve(y_test[:,i], pred_test_v[:,i], pos_label=1)
    dic_res[diseases[i]]=[metrics.auc(fpr, tpr),metrics.auc(recall,precision),metrics.average_precision_score(y_test[:,i],pred_test_v[:,i]),y_test[y_test[:,i]==1].shape[0]/y_test.shape[0]*100]

In [ ]:
'''Store results'''
p=pd.DataFrame(dic_res).T
p.columns=["AUC ROC test","AUC PR test","Average precision score","prevalence in %"]
p.to_csv(path_for_files+"Testc822_mtl_362_l2_dropout_test.csv")